## Azure ML - Sample Batch Prediction Pipeline
This notebook demonstrates creation and execution of an Azure ML pipeline designed to load data from a remote source, to make predictions against that data using a previously registered ML model, and finally save that data to an external sink where it can be consumed by other LOB apps.

### Import Required Packages

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData, PipelineEndpoint
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig

### Connect to Azure ML Workspace, Provision Compute Resources, and get References to Datastores
Connect to workspace using config associated config file. Get a reference to you pre-existing AML compute cluster or provision a new cluster to facilitate processing. Finally, get references to your default blob datastore.

In [ ]:
#Connect to AML Workspace
ws = Workspace.from_config()

#Select AML Compute Cluster
cpu_cluster_name = 'cpucluster'

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found an existing cluster, using it instead.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           min_nodes=0,
                                                           max_nodes=1)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)
    
#Get default datastore
default_ds = ws.get_default_datastore()

### Get Sample Toy Dataset and Save to Default (Blob) Datastore
Load data into a pandas dataframe directly from Scikit-Learn and save as a CSV to predefined location inside the default Azure ML datastore.

Note: These data can be written to any AML-linked datastore using the SDK commands shown below.

In [ ]:
# Load source data - using iris dataset from scikit below
from sklearn.datasets import load_iris
import pandas as pd
import os
import shutil

data = load_iris()

input_df = pd.DataFrame(data.data, columns = data.feature_names)

os.makedirs('./tmp', exist_ok=True)
input_df.to_csv('./tmp/iris_data.csv', index=False)

default_ds.upload(src_dir='./tmp',
                 target_path='iris_data_scoring',
                 overwrite=True)

shutil.rmtree('./tmp')

### Create Run Configuration
The `RunConfiguration` defines the environment used across all python steps. You can optionally add additional conda or pip packages to be added to your environment. [More details here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py). In this example we retrieve the `Environment` which was used for model training to ensure consistency.

In [ ]:
run_config = RunConfiguration()
run_config.docker.use_docker = True
run_config.environment = Environment.get(ws, 'sample_env')

### Define Output Datasets
Below we define the configuration for datasets that will be passed between steps in our pipeline. Note, in all cases we specify the datastore that should hold the datasets and whether they should be registered following step completion or not. This can optionally be disabled by removing the register_on_complete() call.

In [ ]:
inferencing_dataset = OutputFileDatasetConfig(name='inferencing_dataset', destination=(default_ds, 'inferencing_dataset/{run-id}')).read_delimited_files().register_on_complete(name='Inferencing_data')
scored_dataset = OutputFileDatasetConfig(name='scored_dataset', destination=(default_ds, 'scored_dataset/{run-id}')).read_delimited_files().register_on_complete(name='Scored_data')

### Define Pipeline Parameters
`PipelineParameter` objects serve as variable inputs to an Azure ML pipeline and can be specified at runtime. Below we specify a pipeline parameter object model_name which will be used to reference the locally trained model that was uploaded and registered within the Azure ML workspace. Multiple pipeline parameters can be created and used. 

In [ ]:
model_name = PipelineParameter(name='model_name', default_value='Iris_Classification')

### Define Pipeline Steps
The pipeline below consists of steps to gather and register data from a remote source, a scoring step where the registered model is used to make predictions on loaded, and a data publish step where scored data can be exported to a remote data source. All of the `PythonScriptSteps` have a corresponding *.py file which is referenced in the step arguments. Also, any `PipelineParameters` defined above can be passed to and consumed within these steps.

In [ ]:
get_inferencing_data_step = PythonScriptStep(
    name='Get Inferencing Data',
    script_name='get_inferencing_data.py',
    arguments=[
        '--inferencing_dataset', inferencing_dataset
    ],
    outputs=[inferencing_dataset],
    compute_target=cpu_cluster,
    source_directory='./pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

score_inferencing_data_step = PythonScriptStep(
    name='Score Inferencing Data',
    script_name='score_inferencing_data.py',
    arguments=[
        '--model_name', model_name,
        '--scored_dataset', scored_dataset
    ],
    inputs=[inferencing_dataset.as_input(name='inferencing_data')],
    outputs=[scored_dataset],
    compute_target=cpu_cluster,
    source_directory='./pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

publish_scored_data_step = PythonScriptStep(
    name='Publish Scored Data',
    script_name='publish_scored_data.py',
    inputs=[scored_dataset.as_input(name='scored_data')],
    compute_target=cpu_cluster,
    source_directory='./pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

### Create Pipeline
Create an Azure ML Pipeline by specifying the steps to be executed. Note: based on the dataset dependencies between steps, exection occurs logically such that no step will execute unless all of the necessary input datasets have been generated.

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[get_inferencing_data_step, score_inferencing_data_step, publish_scored_data_step])

### Optional: Trigger a Pipeline Execution from the Notebook
You can create an Experiment (logical collection for runs) and submit a pipeline run directly from this notebook by running the commands below

In [ ]:
experiment = Experiment(ws, 'sample-inferencing-pipeline-run')
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)

### Create a Published PipelineEndpoint
Once we have created our pipeline we will look to retrain our model periodically as new data becomes available. By publishing our pipeline to a `PipelineEndpoint` we can iterate on our pipeline definition but maintain a consistent REST API endpoint. 

In [ ]:
from azureml.pipeline.core import PipelineEndpoint

def published_pipeline_to_pipeline_endpoint(
    workspace,
    published_pipeline,
    pipeline_endpoint_name,
    pipeline_endpoint_description="Endpoint to my pipeline",
):
    try:
        pipeline_endpoint = PipelineEndpoint.get(
            workspace=workspace, name=pipeline_endpoint_name
        )
        print("using existing PipelineEndpoint...")
        pipeline_endpoint.add_default(published_pipeline)
    except Exception as ex:
        print(ex)
        # create PipelineEndpoint if it doesn't exist
        print("PipelineEndpoint does not exist, creating one for you...")
        pipeline_endpoint = PipelineEndpoint.publish(
            workspace=workspace,
            name=pipeline_endpoint_name,
            pipeline=published_pipeline,
            description=pipeline_endpoint_description
        )


pipeline_endpoint_name = 'Classification Model Inferencing Pipeline'
pipeline_endpoint_description = 'Sample pipeline for retrieving and scoring data using a classification model based on the Iris Setosa dataset'

published_pipeline = pipeline.publish(name=pipeline_endpoint_name,
                                     description=pipeline_endpoint_description,
                                     continue_on_step_failure=False)

published_pipeline_to_pipeline_endpoint(
    workspace=ws,
    published_pipeline=published_pipeline,
    pipeline_endpoint_name=pipeline_endpoint_name,
    pipeline_endpoint_description=pipeline_endpoint_description
)